## SVD Development


In [1]:
import pyJvsip as pv
from math import sqrt
from svdBidiag import svdBidiag


[ 0.617-0.147i -0.109-0.068i  0.021-0.137i  0.514+0.308i  0.801+0.501i;
 -0.102-0.608i -0.718-0.443i -0.242+0.354i  0.182-1.061i  0.050-0.726i;
  0.664-0.858i -0.623-0.531i -0.261-0.140i -0.216-0.678i -0.377-0.522i;
  0.585-0.611i -0.323+0.219i  0.829+1.505i  0.567+0.668i -0.949-0.322i;
 -0.750+0.447i  0.908-1.522i  0.430+0.775i  0.351-0.299i -0.906-0.329i;
 -0.062+0.052i -0.243-0.454i -0.078+0.586i  1.053-0.065i -0.261+0.667i]

[ 0.617-0.147i -0.109-0.068i  0.021-0.137i  0.514+0.308i  0.801+0.501i;
 -0.102-0.608i -0.718-0.443i -0.242+0.354i  0.182-1.061i  0.050-0.726i;
  0.664-0.858i -0.623-0.531i -0.261-0.140i -0.216-0.678i -0.377-0.522i;
  0.585-0.611i -0.323+0.219i  0.829+1.505i  0.567+0.668i -0.949-0.322i;
 -0.750+0.447i  0.908-1.522i  0.430+0.775i  0.351-0.299i -0.906-0.329i;
 -0.062+0.052i -0.243-0.454i -0.078+0.586i  1.053-0.065i -0.261+0.667i]

('check %e.4', 1.4411674438125916e-06)
[ 1.855  1.320  1.707  1.220  2.023]

[ 1.242  1.265  1.330  1.609]



In [2]:
def svdCorners(f):
    j=f.length-1
    while  j > 0 and f[j] == 0.0:
        j-=1
    if(j == 0 and f[j] == 0.0):
        i=0
    else:
        i = j
        while i > 0 and f[i] != 0.0:
            i -= 1          
        if i==0 and f[0] == 0.0:
            i = 1
            j += 2
        elif i == 0 and f[0] != 0.0:
            i = 0
            j += 2
            
        else:
            i += 1
            j += 2
    return (i,j)

In [3]:
def givensCoef(x1, x2): #defined for SVD iteration when x1 and x2 are real
    a=abs(x1);b=abs(x2)
    if x2 == 0.0:
        return(1.0,0.0,x1)  
    elif a < b:
        t = x1/x2
        t = b * sqrt(1.0 + t * t)
    else:
        t=x2/x1
        t = a * sqrt(1.0 + t * t)
    if (x1 == 0.0):
        c=0.0
        if x2 < 0:
            s = -1.0
        else:
            s = 1.0
        return(c,s,t)
    else:
        s = x2/t
        c = abs(x1)/t
        if x1 < 0:
            s = -s
            t=-t        
        return(c,s,t)
    
def prodG(L,i,j,c,s):
    a1= L.colview(i)
    a2= L.colview(j)
    t    = c * a1 + s * a2
    a2[:]= c * a2 - s * a1
    a1[:]= t
    
def gtProd(i,j,c,s,R):
    a1= R.rowview(i)
    a2= R.rowview(j)
    t= c * a1 + s * a2
    a2[:]=c * a2 - s * a1
    a1[:]=t

In [4]:
#zero row
def zeroRow(L,d,f):
    n = d.length
    xd=d[0]
    xf=f[0]
    c,s,r=givensCoef(xd,xf)
    if n == 1:
        d[0]=r
        f[0]=0.0
        prodG(L,n,0,c,s)
    else:
        d[0]=r
        f[0]=0.0
        xf=f[1]
        t= -xf * s; xf *= c
        f[1]=xf
        prodG(L,1,0,c,s);
        for i in range(1,n-1):
            xd=d[i]
            c,s,r=givensCoef(xd,t)
            prodG(L,i+1,0,c,s)
            d[i]=r
            xf=f[i+1]
            t=-xf * s; xf *= c
            f[i+1]=xf
        xd=d[n-1]
        c,s,r=givensCoef(xd,t)
        d[n-1]=r
        prodG(L,n,0,c,s)

In [5]:
#zero column
def zeroCol(d,f,R):
    n = f.length
    if n == 1:
        c,s,r=givensCoef(d[0],f[0]);
        d[0]=r;
        f[0]=0.0;
        gtProd(0,1,c,s,R)
    elif n == 2:
        xd=d[1]
        xf=f[1]
        c,s,r=givensCoef(xd,xf)
        d[1]= r
        f[1]=0.0
        xf=f[0]
        t= -xf * s; xf *= c;
        f[0]=xf;
        gtProd(1,2,c,s,R);
        xd=d[0]
        c,s,r=givensCoef(xd,t);
        d[0]=r;
        gtProd(0,2,c,s,R);
    else:
        i=n-1; j=i-1; k=i
        xd=d[i]
        xf=f[i]
        c,s,r=givensCoef(xd,xf);
        xf=f[j]
        d[i]=r;
        f[i]=0.0;
        t=-xf*s; xf*=c
        f[j]=xf;
        gtProd(i,k+1,c,s,R);
        while i > 1:
            i = j; j = i-1;
            xd=d[i]
            c,s,r=givensCoef(xd,t);
            d[i]=r;
            xf=f[j]
            t= -xf * s; xf *= c;
            f[j]=xf;
            gtProd(i,k+1,c,s,R);
        xd=d[0]
        c,s,r=givensCoef(xd,t)
        d[0]=r
        gtProd(0,k+1,c,s,R)

In [6]:
#zero find
def zeroFind(d,eps0):
    j = d.length
    xd = d[j-1]
    while xd > eps0:
        if j > 1:
            j -= 1
            xd=d[j-1]
        elif j==1:
            j -= 1
            return 0 # we return here if no zero value was found
    d[j-1]=0.0 # if we get here j > 1
    return j

In [7]:
def svdMu(d2,f1,d3,f2, eps0):
    cu=d2 * d2 + f1 * f1
    cl=d3 * d3 + f2 * f2
    cd = d2 * f2;
    T = (cu + cl)
    D = 4*(cu * cl - cd * cd)/(T*T)
    if D >= 1.0:
        root = 0.0
    else:
        root = T * sqrt(1.0 - D)
    lambda1 = (T + root)/(2.)
    lambda2 = (T - root)/(2.) 
    c1=abs(lambda1-cl)
    c2=abs(lambda2-cl)
    if(root == 0.0):
        if f2 < (d2 + d3) * eps0:
            f2=0.0
    if(c1 < c2):
        mu = lambda1
    else:
        mu = lambda2
    return (mu, f2)

In [8]:
# svd step
def svdStep(L,d,f,R, eps0):
    n = d.length
    mu=0.0; x1=0.0; x2=0.0; t=0.0
    d2=0.0; f1=0.0; d3=0.0; f2=0.0
    assert n > 1, 'Error. d.length for svdStep must be at least 2'
    if n >= 3:
        d2=d[n-2];f1= f[n-3];d3 = d[n-1];f2= f[n-2]
    else: #n == 2
        d2=d[0]; d3=d[1]; f1=0.0; f2=f[0]
    mu, f2 = svdMu(d2, f1, d3, f2, eps0)
    if(f2 == 0.0):
        f[n-2] = 0.0
    x1=d[0]
    x2 = x1 * f[0]
    x1 *= x1; x1 -= mu
    c,s,r=givensCoef(x1,x2)
    x1=d[0];x2=f[0]
    f[0]=c*x2-s*x1
    d[0]=c*x1+s*x2
    t=d[1]
    d[1] *= c
    t *= s
    gtProd(0,1,c,s,R)
    for i in range(n-2):
        j=i+1; k=i+2
        c,s,r = givensCoef(d[i],t)
        d[i]=r
        x1=d[j]*c
        x2=f[i]*s
        t= x1 - x2
        x1=f[i] * c
        x2=d[j] * s 
        f[i] = x1+x2
        d[j] = t
        
        x1=f[j]
        t=s * x1
        f[j]=x1*c
        prodG(L,i, j, c, s)
        c,s,r=givensCoef(f[i],t)
        f[i]=r
        x1=d[j]; x2=f[j]
        d[j]=c * x1 + s * x2; f[j]=c * x2 - s * x1
        x1=d[k]
        t=s * x1; d[k]=x1*c
        gtProd(j,k, c, s,R)
    i=n-2; j=n-1
    c,s,r = givensCoef(d[i],t)
    d[i] = r
    x1=d[j]*c; x2=f[i]*s
    t=x1 - x2
    x1 = f[i] * c; x2=d[j] * s
    f[i] = x1+x2
    d[j] = t
    prodG(L,i, j, c, s)

In [9]:
# phase check
# When this is called d and f are assumed strictly real
def phaseCheck(L,d,f,R,eps0):
    nf=f.length;
    for i in range(d.length):
        ps=d[i]
        m = abs(ps)
        if(m >= eps0) and ps < 0.0:
            L.colview(i).neg
            d[i]=m;
            if (i < nf):
                f[i] = -f[i]
        elif m < eps0:
            d[i]=0.0
    # at this point d is strictly >= 0
    # below we check f for zeros
    for i in range(nf):
        if abs(f[i]) < (d[i] + d[i+1]) * eps0:
            f[i]=0.0
    

In [10]:
def svdIteration(L0,d0,f0,R0,eps0):
    cntr=0.0
    # arbitrary limit based on experience (we normally converge before this)
    maxcntr = 5 * d0.length 
    while (cntr < maxcntr):
        cntr += 1
        i,j=svdCorners(f0)
        if (j == 0): #Done
            break
        d=d0[i:j]; f=f0[i:j-1]
        L=L0[:,i:j];R=R0[i:j,:]
        
        n=f.length
        k=zeroFind(d,eps0)
        if (k > 0 ):
            k=k-1;
            if(d[n] == 0.0):
                zeroCol(d,f,R);   
            else:
                L=L[:,k:]
                d.putlength(d.length-k+1)
                d.putoffset(d.offset+k+1)
                f.putlength(f.length - k)
                f.putoffset(f.offset + k)
                zeroRow(L,d,f)
        else:
            svdStep(L,d,f,R, eps0)
        phaseCheck(L0,d0,f0,R0,eps0)
    return (L0,d0,R0)

In [11]:
def svdSort(L,d,R):
    n=d.length;
    indx = d.sort('BYVALUE','DESCENDING')
    L[:,:n].permute(indx.copy,'COL')
    R.permute(indx,'ROW')
    return L,d,R

#### Example (test)

In [12]:
A=pv.create('cmview_d',6,6).randn(8)
fmt = '%5.4f'
A.mprint(fmt)
L,d,R = A.svd
print('results from JVSIP svd calculation')
print('d'); d.mprint(fmt)
print('L'); L.mprint(fmt)
print('R'); R.mprint(fmt)
D = A.empty.fill(0.0)
pv.copy(d,D.diagview(0).realview)
L.prod(D).prod(R).mprint(fmt)
B=A.empty.fill(0.0)
B.randn(8)
if 'cmview' in B.type:
    b=B.realview.diagview
else:
    b=B.diagview
L,d,f,R,eps=svdBidiag(B,1E-10)
print('results after svdBidiag calculation')
print('d'); d.mprint(fmt)
print('L'); L.mprint(fmt)
print('R'); R.mprint(fmt)
L1,d1,R1=svdIteration(L.copy,d.copy,f.copy,R.copy,1E-10)
print('results after svdIteration calculation')
print('d'); d1.mprint(fmt)
print('L'); L1.mprint(fmt)
print('R'); R1.mprint(fmt)
svdSort(L1, d1, R1)
print('results after svdSort calculation')
print('d'); d1.mprint(fmt)
print('L'); L1.mprint(fmt)
print('R'); R1.mprint(fmt)
D = A.empty.fill(0.0)
pv.copy(d1,D.diagview(0).realview)
L1.prod(D).prod(R1.herm).mprint(fmt)


[-0.0639-0.1735i -0.2647+0.9114i -0.3530+0.4305i -0.0299-0.9771i -0.2578+0.3942i -0.9996+0.7310i;
  1.2451+0.9822i -0.8168+0.0997i  0.5731+0.3959i -0.0030+0.2353i -0.3665-0.6564i  0.6063-0.9247i;
 -0.7686+1.0487i  1.4887+0.6151i  0.2907+0.6947i -0.3759+0.5722i  0.3870+0.3083i -0.2531-0.5128i;
  0.0935+0.0792i  0.1603+0.2637i -0.3607-0.8815i -0.3867+1.1440i -0.6202-0.3463i  0.7403+0.1240i;
  0.5802-0.3386i -0.4374+0.8039i  0.0109-0.0618i  0.5534-0.3221i -0.1440-0.3925i -0.2355-0.1393i;
  0.4339+0.0660i -0.4843-0.7487i -0.7696-0.6345i  1.1468+0.1345i -0.2555-0.9663i -0.6679+0.2204i]

results from JVSIP svd calculation
d
[ 3.5972  2.3905  2.1332  1.3780  0.7337  0.1795]

L
[-0.1375-0.4417i  0.1731+0.0451i -0.2284-0.2371i  0.0234-0.4372i  0.2970+0.4147i -0.3426-0.2763i;
  0.3157+0.1788i -0.6104-0.4066i -0.2069-0.3197i -0.1345-0.1268i -0.1635+0.0200i -0.3508+0.0323i;
 -0.4220+0.3227i -0.1664+0.3487i  0.0944-0.4483i  0.0207-0.4706i -0.1201-0.0096i  0.2620+0.2314i;
  0.0381+0.1534i -0.2894-0.

In [13]:
A=pv.create('cmview_d',6,6).randn(8)
fmt = '%5.4f'
A.mprint(fmt)
L,d,R = A.svd

[-0.0639-0.1735i -0.2647+0.9114i -0.3530+0.4305i -0.0299-0.9771i -0.2578+0.3942i -0.9996+0.7310i;
  1.2451+0.9822i -0.8168+0.0997i  0.5731+0.3959i -0.0030+0.2353i -0.3665-0.6564i  0.6063-0.9247i;
 -0.7686+1.0487i  1.4887+0.6151i  0.2907+0.6947i -0.3759+0.5722i  0.3870+0.3083i -0.2531-0.5128i;
  0.0935+0.0792i  0.1603+0.2637i -0.3607-0.8815i -0.3867+1.1440i -0.6202-0.3463i  0.7403+0.1240i;
  0.5802-0.3386i -0.4374+0.8039i  0.0109-0.0618i  0.5534-0.3221i -0.1440-0.3925i -0.2355-0.1393i;
  0.4339+0.0660i -0.4843-0.7487i -0.7696-0.6345i  1.1468+0.1345i -0.2555-0.9663i -0.6679+0.2204i]



In [14]:
def jsvd(A,eps0):
    L,d,f,R,eps0 = svdBidiag(A,eps0)
    L,d,R=svdIteration(L,d,f,R,eps0)
    return (svdSort(L,d,R))
def jsvdTest(A):
    L,d,R = jsvd(A,1E-15)
    Ac=A.empty.fill(0.0)
    if 'cmview' in Ac.type:
        Ac.realview.diagview(0)[:]=d
    else:
        Ac.diagview(0)[:]=d
    d.mprint('%.4f')
    e=((A-L.prod(Ac).prod(R)).normFro)/(A.normFro)
    print('Error check %.5e'%e)

In [15]:
A.randn(8)
jsvdTest(A)

[ 3.5972  2.3905  2.1332  1.3780  0.7337  0.1795]

Error check 7.94006e-16
